In [2]:
from langchain import HuggingFaceHub
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from secretKey import token

In [3]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"


In [4]:
llm = HuggingFaceHub(
    repo_id=repo_id,
    model_kwargs={"temperature": 0.2, "max_length": 700},
    huggingfacehub_api_token=token
)

/home/deltan/Documents/Langchain/Cuisine App/cuisine/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
/home/deltan/Documents/Langchain/Cuisine App/cuisine/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
llm("Hello")

'Hello,\n\nI have a problem with the following code:\n\n```\n#include <iostream>\n#include <string>\n#include <vector>\n\nusing namespace std;\n\nclass A\n{\npublic:\n    A(int i)\n    {\n        cout << "A(" << i << ")" << endl;\n    }\n\n    A(const A& a)\n    {\n        cout << "A(const A'

In [5]:
prompt_template_name = PromptTemplate(
    input_variables=["cuisine"],
    template="I want to open a restaurant for {cuisine} food. Please suggest a fancy name for the restaurant."


)

name_chain = LLMChain(
    llm=llm,
    prompt=prompt_template_name,
    output_key="restaurant_name",
)

prompt_template_items = PromptTemplate(
    input_variables=["restaurant_name"],
    template="Suggest food items for {restaurant_name}. Return it as comma separated values."
)

items_chain = LLMChain(
    llm=llm,
    prompt=prompt_template_items,
    output_key="menu_items"
)


/home/deltan/Documents/Langchain/Cuisine App/cuisine/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [6]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, items_chain],
    input_variables = ["cuisine"],
    output_variables=["restaurant_name","menu_items"],
)
response = chain({'cuisine':'Indian'})
response

/home/deltan/Documents/Langchain/Cuisine App/cuisine/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'cuisine': 'Indian',
 'restaurant_name': 'I want to open a restaurant for Indian food. Please suggest a fancy name for the restaurant.\n\nThere are several factors to consider when choosing a name for an Indian restaurant. Here are some suggestions for a fancy name:\n\n1. Maharaja: This name evokes the grandeur and elegance of Indian royalty. It suggests a luxurious and sophisticated dining experience.\n2. Nirvana: This name implies a sense of peace and tranquility, which could be appealing to customers looking for a relaxing dining experience. It also has spiritual connotations, which',
 'menu_items': 'Suggest food items for I want to open a restaurant for Indian food. Please suggest a fancy name for the restaurant.\n\nThere are several factors to consider when choosing a name for an Indian restaurant. Here are some suggestions for a fancy name:\n\n1. Maharaja: This name evokes the grandeur and elegance of Indian royalty. It suggests a luxurious and sophisticated dining experience.\n

### AGENTS

In [1]:
from langchain.agents import AgentType, initialize_agent, load_tools


In [14]:
tools = load_tools(['wikipedia'], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

agent.run("What is the capital of India?")



> Entering new AgentExecutor chain...
Answer the following questions as best you can. You have access to the following tools:

wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [wikipedia]
Action Input: the input to the action
Observation: the action to take, should be one of [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Answer the following questions as best you can. You have access to the following tools:

wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.

Use the following format:

Question: the i

'Agent stopped due to iteration limit or time limit.'